# Markov Chains

Math 321<br>
October 26, 2017

A markov chain is a model that describes a sequence of events where the probability of an event is determined only by the previous event in the sequence. 

A markov chain can be described by a stochastic matrix called a transition matrix. A stochastic matrix is a matrix where the columns sum to 1.

In the context of a markov chain with $n$ possible states, a $n \times n$ transition matrix $A$ can be constructed where the probability of moving to state $j$ given that we are in state $i$ is given by $A_{ji}$, that is, the value in row $j$ and column $i$.

Let's do an example.

## Weather Forcasting

We can forecast the weather with a markov chain, given that we know some information up front.

Let's say that in a certain city, the weather has 4 basic states: cold, mild, warm, and hot. We'll assign each state a number as follows: cold: 0, mild: 1, warm: 2, and hot: 3.

Through historical observation of the weather, let's say that we figured out that, given that today is cold, the probabilty of it being cold tomorrow is 0.5; of it being mild is 0.3, warm is 0.2, and hot is 0. These, and the other transition probabilities, are given by the transition matrix below.

$$
\begin{pmatrix}
0.5 & 0.3 & 0.1 & 0\\
0.3 & 0.3 & 0.3 & 0.3\\
0.2 & 0.3 & 0.4 & 0.5\\
0 & 0.1 & 0.2 & 0.2
\end{pmatrix}
$$

Using this matrix, let's write a function to forecast the weather for $n$ days, given that today is mild. 

In [40]:
import numpy as np
from numpy.linalg import norm
import pdb

def four_state_forecast(days):
    """Run a simulation for the weather over the specified number of days,
    with mild as the starting state, using the four-state Markov chain.
    Return a list containing the day-by-day results, not including the
    starting day.

    Examples:
        >>> four_state_forecast(3)
        [0, 1, 3]
        >>> four_state_forecast(5)
        [2, 1, 2, 1, 1]
    """
    M = np.array([[.5,.3,.1,0],[.3,.3,.3,.3],[.2,.3,.4,.5],[0,.1,.2,.2]])
    prev_day = 1 # mild
    forecast = []
    for k in range(days):
        today = np.nonzero(np.random.multinomial(1, M[:,prev_day]))[0][0]
        forecast.append(today)
        prev_day = today
    return forecast

Now let's predict the weather for the next 5 days.

In [41]:
four_state_forecast(5)

[1, 1, 2, 3, 1]

In [42]:
M = np.array([[.5,.3,.1,0],[.3,.3,.3,.3],[.2,.3,.4,.5],[0,.1,.2,.2]])

## Generating Text

Another cool (though not very useful) application of markov chains is simulating speech in the style of some text.

Given a sample of text, be it from a book, or lines of dialogue in a movie, or something else, we can construct a transition matrix from the probability of occurance of each word in the text.

In [43]:
class SentenceGenerator(object):
    """Markov chain creator for simulating bad English.

    Attributes:
        transition ((n,n) ndarray): transition matrix for the markov chain
        states (list of length n): words in the order used by the transition matrix

    Example:
        >>> yoda = SentenceGenerator("Yoda.txt")
        >>> print(yoda.babble())
        If once you trained, gone am I do not, miss them do you?
    """
    def __init__(self, filename):
        """Read the specified file and build a transition matrix from its
        contents. We will assume that the file has one complete sentence
        written on each line.
        """
        word_set = set()
        with open(filename, 'r') as f:
            for sentence in f:
                word_set.update(sentence.strip().split())
        n = len(word_set)
        
        self.transition = np.zeros((n+2,n+2))
        self.states = ["$tart"] + list(word_set) + ["$top"]

        with open(filename, 'r') as f:
            for sentence in f:
                words = sentence.strip().split()
                
                first_word_idx = self.states.index(words[0])
                last_word_idx = self.states.index(words[-1])
                
                self.transition[first_word_idx, 0] += 1
                self.transition[n+1, last_word_idx] += 1

                for k in range(len(words) - 1):
                    current_word_idx = self.states.index(words[k])
                    next_word_idx = self.states.index(words[k+1])
                    self.transition[next_word_idx, current_word_idx] += 1
        
        # You can't leave the $top state once you're in it
        self.transition[n+1, n+1] = 1
        # Make each column sum to 1
        self.transition = self.transition / np.sum(self.transition, axis=0)

    def babble(self):
        """Produce a string of text using the markov chain model produced
        by the transition matrix created in the constructor.
        
        When the '$top' state is reached, stop transitioning and terminate the
        sentence.
        """
        sentence = []
        state = 0
        while state != len(self.states) - 1:
            draw = np.random.multinomial(1, self.transition[:,state])
            state = np.nonzero(draw)[0][0]
            sentence.append(self.states[state])
        return ' '.join(sentence[:-1])

In [44]:
yoda = SentenceGenerator("Vol-2/MarkovChains/yoda.txt")

In [53]:
yoda.babble()

'A Jedi craves not that which you should not.'